In [1]:
import pandas as pd
from transformers import TFBertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

D:\Anaconda\envs\stock_sentiment_predictor\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 다양한 주체가 있는 문장을 추가적으로 Fine-tuning시키기

- 뉴스 감정 예측 모델의 성능을 향상시키기 위해, **다양한 주체**가 포함된 애매한 문장을 추가적으로 학습하는 Fine-tuning을 진행
- Fine-tuning 이전에는 아래와 같은 문장에서 감정 예측 성능이 부족했으며, 이를 보완하기 위해 학습 데이터에서 **다양한 주체가 포함된 문장**들을 선별하여 Fine-tuning 데이터셋을 생성
- 하지만 선별된 데이터의 양이 적어 학습에 적합하지 않았기 때문에, **K-TACC**를 사용하여 데이터 증강을 진행

---

### Fine-tuning 이전 성능

| 문장                                                                         | 실제 감정 | 예측 감정 |
|----------------------------------------------------------------------------|-------|-------|
| 삼성전자가 실적 발표에서 긍정적인 결과를 보였으나, SK하이닉스는 부진했다...                               | 부정    | 중립    |
| SK하이닉스는 실적 상승을 기록했지만 삼성전자는 다소 실망스러운 실적을 발표했다...                            | 긍정    | 부정    |
| SK하이닉스의 실적 발표 발표에서 클라우드 부문의 성장 둔화가 우려를 불러일으켰다, 아마존는 반도체 부문 시장에서 강세를 보였다... | 부정    | 긍정    |

---

### Fine-tuning 이후 성능

| 문장                                                                         | 실제 감정 | 예측 감정 |
|----------------------------------------------------------------------------|-------|-------|
| 삼성전자가 실적 발표에서 긍정적인 결과를 보였으나, SK하이닉스는 부진했다...                               | 부정    | 부정    |
| SK하이닉스는 실적 상승을 기록했지만 삼성전자는 다소 실망스러운 실적을 발표했다...                            | 긍정    | 긍정    |
| SK하이닉스의 실적 발표 발표에서 클라우드 부문의 성장 둔화가 우려를 불러일으켰다, 아마존는 반도체 부문 시장에서 강세를 보였다... | 부정    | 부정    |

---

Fine-tuning을 통해 애매한 주체가 포함된 문장에서도 모델의 감정 예측 성능 개선


In [2]:
# 모델 경로와 토크나이저 경로 설정
MODEL_PATH = "../sentiment_analysis_model"  # 모델이 저장된 경로
MODEL_NAME = "klue/bert-base"  # 모델 이름

tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
model = TFBertForSequenceClassification.from_pretrained(MODEL_PATH)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ../sentiment_analysis_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [3]:
# 다양한 주체가 있는 데이터 불러오기
dataset = pd.read_csv("../data/augmented_subject_focus_data.csv", encoding="utf-8", header=None, names=['description', 'sentiment'])

In [4]:
# 데이터 셋 요약 확인
print('데이터 요약')
dataset.info()

# 'description' 컬럼을 문자열 형식으로 변환
dataset['description'] = dataset['description'].astype(str)

# 데이터 셔플
dataset = dataset.sample(frac=1).reset_index(drop=True)

# 셔플 된 데이터 확인
print(dataset.head())

# 데이터 전처리
def encode_data(data):
    return tokenizer(
        data['description'].tolist(),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="tf"
    )

# 학습 데이터와 테스트 데이터 분리
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

# 데이터 전처리
train_encodings = encode_data(train_data)
val_encodings = encode_data(val_data)

데이터 요약
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  537 non-null    object
 1   sentiment    537 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.5+ KB
                                         description  sentiment
0  삼성전자는 실적 부진에도 불구하고 금리 추가 인하 기대감으로 상승했지만 SK하이닉스...          0
1  반면 아마존은 글로벌 경제 불확실성 속에서도 전과 다름없이 긍정적인 실적을 기록했지...          0
2  삼성전자는 부진했지만 같은 52시간 규제안 속에서도 SK하이닉스는 최고의 실적을 냈...          2
3  삼성전자의 경우 HBM 분야의 기술 경쟁력이 뒤쳐진 상태이기에 범용 메모리반도체 시...          2
4  중국의 반도체 창신메모리CXMT는 D램 생산 기술 분야에서 삼성전자와 SK하이닉스 ...          0


In [5]:
# 감정 레이블
train_labels = train_data['sentiment'].values
val_labels = val_data['sentiment'].values

# TensorFlow Dataset 생성
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings), train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings), val_labels
))

# 배치 처리 및 데이터 셔플
train_dataset = train_dataset.shuffle(len(train_data)).batch(16)
val_dataset = val_dataset.batch(16)

# 콜백 함수 정의 (EarlyStopping)
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=0.001, patience=2),
]

# 모델 학습 준비 (Adam 옵티마이저, 손실 함수 설정)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# 모델 학습
model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=callbacks)

# 학습된 모델 저장
model.save_pretrained('../subject_focus_finetuned_model')
tokenizer.save_pretrained("../subject_focus_finetuned_model")

Epoch 1/10
27/27 [==============================] - 33s 500ms/step - loss: 1.5113 - accuracy: 0.5175 - val_loss: 0.7585 - val_accuracy: 0.6389
Epoch 2/10
27/27 [==============================] - 10s 372ms/step - loss: 0.7629 - accuracy: 0.6550 - val_loss: 0.5351 - val_accuracy: 0.7685
Epoch 3/10
27/27 [==============================] - 10s 373ms/step - loss: 0.5607 - accuracy: 0.7855 - val_loss: 0.3828 - val_accuracy: 0.8796
Epoch 4/10
27/27 [==============================] - 10s 379ms/step - loss: 0.3903 - accuracy: 0.8625 - val_loss: 0.3121 - val_accuracy: 0.9259
Epoch 5/10
27/27 [==============================] - 10s 374ms/step - loss: 0.2819 - accuracy: 0.9138 - val_loss: 0.2632 - val_accuracy: 0.9167
Epoch 6/10
27/27 [==============================] - 10s 377ms/step - loss: 0.2114 - accuracy: 0.9394 - val_loss: 0.2003 - val_accuracy: 0.9537
Epoch 7/10
27/27 [==============================] - 10s 373ms/step - loss: 0.1622 - accuracy: 0.9510 - val_loss: 0.1448 - val_accuracy: 0.9444

('../subject_focus_finetuned_model\\tokenizer_config.json',
 '../subject_focus_finetuned_model\\special_tokens_map.json',
 '../subject_focus_finetuned_model\\vocab.txt',
 '../subject_focus_finetuned_model\\added_tokens.json')